In [1]:
#!pip install pyxlsb

README: 
Objectif récuppérer les loyers du parcs privé par communes et récuppérer les contours géographiques des communes. On exporte ensuite une nouvelle base de données. 

In [2]:
import pandas as pd
import pyxlsb
import geopandas as gpd


In [3]:
df = pd.read_csv("indicateurs-loyers-appartements.csv", encoding= 'unicode_escape', sep = ";")

In [4]:
df

,id_zone,INSEE,LIBGEO,DEP,REG,EPCI,TYPPRED,loypredm2,lwr.IPm2,upr.IPm2,R2adj,NBobs_maille,NBobs_commune
0,2362,1001,L'Abergement-Clémenciat,1,84,200069193,maille,"9,372335348","7,409663525","11,85487972","0,774249278",701,2
1,2010,1002,L'Abergement-de-Varey,1,84,240100883,maille,"8,63555202","6,72107751","11,09535764","0,719598662",861,0
2,1456,1004,Ambérieu-en-Bugey,1,84,240100883,commune,"10,07450708","7,865307059","12,90422511","0,670160882",2156,2156
3,2362,1005,Ambérieux-en-Dombes,1,84,200042497,maille,"9,372335348","7,409663525","11,85487972","0,774249278",701,81
4,2055,1006,Ambléon,1,84,200040350,maille,"8,96695486","7,101593346","11,3222872","0,752823051",842,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35436,1802,97420,Sainte-Suzanne,974,4,249740119,commune,"10,67356968","8,791530189","12,95850521","0,788143839",847,411
35437,1802,97421,Salazie,974,4,249740093,maille,"10,38745938","8,545191709","12,62690364","0,788143839",847,11
35438,1608,97422,Le Tampon,974,4,249740085,commune,"10,81548488","8,696332802","13,45103918","0,81541405",3237,3237
35439,1607,97423,Les Trois-Bassins,974,4,249740101,maille,"12,34315971","8,597802405","17,72006199","0,460731322",1669,90


On ne garde que le code INSEE et le loyer supérieur par m^2

In [5]:
df_private = df[["INSEE", "upr.IPm2"]]

On renomme la colonne INSEE en codgeo pour pouvoir la merger avec la carte des contours des villes. 

In [6]:
df_private.rename(columns = {'INSEE':'codgeo'}, inplace = True)
df_private = gpd.GeoDataFrame(df_private)


/tmp/ipykernel_1362/3783642682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_private.rename(columns = {'INSEE':'codgeo'}, inplace = True)


In [7]:
"""
On rajoute un 0 devant le code INSEE quand il ne fait que quatre chiffres (pour uniformiser avec l'autre base). 
"""
for i in range(0,df_private.shape[0]):
    if  len(df_private["codgeo"][i]) == 4: 
            df_private["codgeo"][i] = "0" +  df_private["codgeo"][i]


In [8]:
df_private = df_private.set_index('codgeo') #on réindex le dataframe avec le code INSEE pour merger après. 

In [9]:
df_private

,upr.IPm2
codgeo,
01001,"11,85487972"
01002,"11,09535764"
01004,"12,90422511"
01005,"11,85487972"
01006,"11,3222872"
...,...
97420,"12,95850521"
97421,"12,62690364"
97422,"13,45103918"


On récuppère les contours géographique des communes.

In [10]:
gdf = gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/fb3580f6-e875-408d-809a-ad22fc418581")

ERROR 1: PROJ: proj_create_from_database: Open of /opt/mamba/share/proj failed


In [11]:
gdf

,codgeo,dep,reg,xcl2154,ycl2154,libgeo,geometry
0,01001,01,84,848241,6563021,L'Abergement-Clémenciat,"POLYGON ((4.90497 46.16079, 4.90280 46.15795, ..."
1,01002,01,84,887495,6548152,L'Abergement-de-Varey,"POLYGON ((5.40440 46.00778, 5.41628 46.00056, ..."
2,01004,01,84,882724,6542583,Ambérieu-en-Bugey,"POLYGON ((5.38637 45.93121, 5.38892 45.93320, ..."
3,01005,01,84,847277,6545791,Ambérieux-en-Dombes,"POLYGON ((4.88657 46.01602, 4.88808 46.01061, ..."
4,01006,01,84,902191,6519791,Ambléon,"POLYGON ((5.59066 45.75997, 5.58396 45.76274, ..."
...,...,...,...,...,...,...,...
34950,97615,976,06,570763,6101694,Pamandzi,"POLYGON ((1.43812 42.01084, 1.44057 42.00601, ..."
34951,97616,976,06,549586,6095929,Sada,"POLYGON ((1.21289 41.95784, 1.20638 41.95494, ..."
34952,97617,976,06,549653,6103182,Tsingoni,"POLYGON ((1.21959 42.05521, 1.21213 42.05005, ..."
34953,16233,16,75,464776,6506602,Mosnac-Saint-Simeux,"POLYGON ((-0.05659 45.62068, -0.04989 45.61714..."


On ne garde que le code INSEE et les données polygoniales. 

In [12]:
gdf_private = gdf[["codgeo", "geometry"]]

In [13]:
gdf_private = gdf_private.set_index('codgeo')

On joint nos deux base à partir de codgeo. 

In [14]:
gdf_privated = pd.concat([df_private,gdf_private], axis = 1, join = "inner")

In [15]:
gdf_privated

,upr.IPm2,geometry
codgeo,,
01001,"11,85487972","POLYGON ((4.90497 46.16079, 4.90280 46.15795, ..."
01002,"11,09535764","POLYGON ((5.40440 46.00778, 5.41628 46.00056, ..."
01004,"12,90422511","POLYGON ((5.38637 45.93121, 5.38892 45.93320, ..."
01005,"11,85487972","POLYGON ((4.88657 46.01602, 4.88808 46.01061, ..."
01006,"11,3222872","POLYGON ((5.59066 45.75997, 5.58396 45.76274, ..."
...,...,...
97420,"12,95850521","POLYGON ((0.37453 42.20905, 0.36151 42.20963, ..."
97421,"12,62690364","POLYGON ((0.30886 42.10112, 0.30640 42.10312, ..."
97422,"13,45103918","POLYGON ((0.14977 41.79845, 0.16024 41.79188, ..."


In [21]:
gdf_privated.to_csv(r'loyer_privé.csv', index = True)